In [1]:
# Loading packages
import numpy as np
import sdeint
import deepdish.io as dd
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
from scipy import signal
import matplotlib.gridspec as gridspec
%matplotlib inline
import time
from IPython.display import set_matplotlib_formats
from ipywidgets import interact, interactive,FloatSlider,IntSlider,ToggleButton
from ipywidgets import Button, HBox, VBox, Text,Select,Layout,Dropdown
set_matplotlib_formats('pdf', 'png')
plt.rcParams['savefig.dpi'] = 75

plt.rcParams['figure.autolayout'] = False
plt.rcParams['figure.figsize'] = 10, 6
plt.rcParams['axes.labelsize'] = 18
plt.rcParams['axes.titlesize'] = 20
plt.rcParams['font.size'] = 16
plt.rcParams['lines.linewidth'] = 2.0
plt.rcParams['lines.markersize'] = 8
plt.rcParams['legend.fontsize'] = 14

plt.rcParams['text.usetex'] = True
plt.rcParams['font.family'] = "sans"
plt.rcParams['font.serif'] = "cm"
#plt.rcParams['text.latex.preamble'] = "\usepackage{subdepth}, \usepackage{type1cm}"

In [2]:
# Setting up the model
from BenincaModel import BenincaModel,Es_normal
#Es_normal['rhs']='RM_forced'
Es_normal["verbose"]=False
m = BenincaModel(Es=Es_normal,Ps='auto/Beninca_set1.hdf5',Vs=None)
p=m.p.copy()
print(p)

{'Tmax': 17.1, 'Tmean': 17.1, 'alpha': 1.0, 'cAB': 0.049, 'cAR': 0.021, 'cBR': 0.018, 'cM': 0.078, 'mA': 0.013, 'mB': 0.003, 'mM': 0.017, 'muAB': 0.036, 'muAR': 0.008, 'muB': 0.015, 'muM': 0.061}


In [3]:
t,sol_const = m.ode_integrate([0.5,0.5,0.5,0.5])

In [4]:
print(sol_const[-1])
init_cond = sol_const[-1]

[0.16326531 0.01749869 0.05468341 0.11538462]


In [5]:
# Erase the initial integration variables to save space
%reset_selective -f "^t$"
%reset_selective -f "^sol_const$"

In [ ]:
peaks=signal.find_peaks(sol_const[:,0]+sol_const[:,1])
print(t[peaks[0]])

In [ ]:
# Plot constant
fig,ax=plt.subplots(1,2,figsize=(4*1.618,6))
gs = gridspec.GridSpec(3, 1)
ax3 = plt.subplot(gs[2:,:])
ax1 = plt.subplot(gs[0, :])#, sharey=ax3, sharex=ax3)
ax2 = plt.subplot(gs[1, :])#, sharey=ax3, sharex=ax3)
ax1.plot(t/365,(sol_const[:,0]+sol_const[:,1])*100.0,'b',label=r'Barnacles')
ax2.plot(t/365,sol_const[:,2]*100.0,'g',label=r'Algae')
ax3.plot(t/365,sol_const[:,3]*100.0,'r',label=r'Mussels')
ax1.set_ylabel(r'Barnacles $[\%]$')
ax1.axes.xaxis.set_ticklabels([])
ax2.axes.xaxis.set_ticklabels([])
#ax3.axes.xaxis.set_ticklabels(np.arange(30,51,1))
ax2.set_ylabel(r'Algae $[\%]$')
ax3.set_ylabel(r'Mussels $[\%]$')
ax3.set_xlabel(r'Time $[years]$')
ax1.set_xlim([0/365,t[-1]/365])
ax2.set_xlim([0/365,t[-1]/365])
ax3.set_xlim([0/365,t[-1]/365])
ax1.stem(t[peaks[0]]/365,(sol_const[:,0]+sol_const[:,1])[peaks[0]]*100.0)
plt.tight_layout()
#plt.savefig('results/Beninca_const_temp.pdf')
#plt.savefig('results/Beninca_const_temp.png')

In [6]:
Tmax=17.5

ito=0.001

int_finish=50*365

# Define ito
parameters={}
parameters['Tmax']=Tmax
m.update_parameters(parameters)
def G(y, t):
    return np.array([[ito,0.0,0.0,0.0],[0.0,ito,0.0,0.0],[0.0,0.0,ito,0.0],[0.0,0.0,0.0,ito]])
step=0.01
tspan = np.arange(0.0, int_finish+step,step)
start=time.time()
result = sdeint.itoint(m.rhs_ode, G, init_cond, tspan)
print("Took ",(time.time()-start)/60.0," minutes")

Took  3.9412644108136496  minutes


In [ ]:
# Erase results !!!!!!!!
%reset_selective -f "^result$"

In [ ]:
# Plot with fft
finish=50*365
trim_yr=30
trim=trim_yr*365
fig,ax=plt.subplots(1,2,figsize=(6*1.618,6),constrained_layout=True)
gs = gridspec.GridSpec(3, 3)
ax3 = plt.subplot(gs[2:,:-1])
ax1 = plt.subplot(gs[0, :-1])#, sharey=ax3, sharex=ax3)
ax2 = plt.subplot(gs[1, :-1])#, sharey=ax3, sharex=ax3)
ax4 = plt.subplot(gs[:, -1])
forcing_tspan = m.Ft(tspan)
plot_forcing_tspan = 10.0*forcing_tspan/np.amax(forcing_tspan)
ax1.plot(tspan/365-trim_yr,(result[:,0]+result[:,1])*100.0,'b',label=r'Barnacles')
ax2.plot(tspan/365-trim_yr,result[:,2]*100.0,'g',label=r'Algae')
ax3.plot(tspan/365-trim_yr,result[:,3]*100.0,'r',label=r'Mussels')
ax1.plot(tspan/365-trim_yr,plot_forcing_tspan,'m:',label=r'forcing',lw=1)
ax2.plot(tspan/365-trim_yr,plot_forcing_tspan,'m:',label=r'forcing',lw=1)
ax3.plot(tspan/365-trim_yr,plot_forcing_tspan,'m:',label=r'forcing',lw=1)
ax1.set_ylabel(r'Barnacles $[\%]$')
ax1.axes.xaxis.set_ticklabels([])
ax2.axes.xaxis.set_ticklabels([])
#ax3.axes.xaxis.set_ticklabels(np.arange(30,51,1))
ax2.set_ylabel(r'Algae $[\%]$')
ax3.set_ylabel(r'Mussels $[\%]$')
ax3.set_xlabel(r'Time $[years]$')
ax1.set_xlim([0,20])
ax2.set_xlim([0,20])
ax3.set_xlim([0,20])
ax1.set_ylim([-10,110])
ax2.set_ylim([-10,110])
ax3.set_ylim([-10,110])
# FFT
forcing = m.Ft(tspan)
#print(len(forcing))
trimfft = int(len(forcing)*(2.0/5.0)) # the index from which to trim the time series to clean transients
#print(trim)
frq = np.fft.fftfreq(forcing[-trimfft:].size,d=0.01/365)
fft_forcing = np.absolute((np.fft.fft(forcing[-trimfft:])))
fft_signal_B  = np.absolute((np.fft.fft(result[-trimfft:,0]+result[-trimfft:,1])))
fft_signal_M  = np.absolute((np.fft.fft(result[-trimfft:,-1])))
normalize_fft = np.amax(fft_signal_B[1:])
ax4.plot(frq[1:],fft_forcing[1:]/np.amax(fft_forcing[1:]),'m:',label=r'forcing')
ax4.plot(frq[1:],fft_signal_B[1:]/normalize_fft,'b',label=r'Barnacles')
ax4.plot(frq[1:],fft_signal_M[1:]/normalize_fft,'r',label=r'Mussels')
ax4.set_xlim([0.1,2.0])
ax4.set_ylim([-0.01,1.2])
ax4.set_xlabel(r'freq $[1/years]$')
#ax[0].legend(loc='upper left')
ax4.legend(loc='upper right')
#ax[0].legend(loc='upper left')
#ax[1].legend(loc='upper left')
plt.savefig('results/Beninca_Ito_and_fft_Tmax{:3.2f}_ito{:5.4f}'.format(Tmax,ito).replace('.','_')+'.pdf')
plt.savefig('results/Beninca_Ito_and_fft_Tmax{:3.2f}_ito{:5.4f}'.format(Tmax,ito).replace('.','_')+'.png')

/home/ohm/anaconda3/lib/python3.7/site-packages/matplotlib/_constrained_layout.py:158: UserWarning: There are no gridspecs with layoutboxes. Possibly did not call parent GridSpec with the figure= keyword
  warnings.warn('There are no gridspecs with layoutboxes. '


In [ ]:
# Plot time series
trim=0*365
finish = int(int_finish*1.0)

fig,ax=plt.subplots(1,2,figsize=(6*1.618,6))
gs = gridspec.GridSpec(3, 3)
ax3 = plt.subplot(gs[2:,:-1])
ax1 = plt.subplot(gs[0, :-1])#, sharey=ax3, sharex=ax3)
ax2 = plt.subplot(gs[1, :-1])#, sharey=ax3, sharex=ax3)
ax4 = plt.subplot(gs[:, -1])
forcing_tspan = m.Ft(tspan)
plot_forcing_tspan = 10.0*forcing_tspan/np.amax(forcing_tspan)
ax1.plot(tspan/365,(result[:,0]+result[:,1])*100.0,'b',label=r'Barnacles')
ax2.plot(tspan/365,result[:,2]*100.0,'g',label=r'Algae')
ax3.plot(tspan/365,result[:,3]*100.0,'r',label=r'Mussels')
ax1.plot(tspan/365,plot_forcing_tspan,'m:',label=r'forcing',lw=1)
ax2.plot(tspan/365,plot_forcing_tspan,'m:',label=r'forcing',lw=1)
ax3.plot(tspan/365,plot_forcing_tspan,'m:',label=r'forcing',lw=1)
ax1.set_ylabel(r'Barnacles $[\%]$')
ax1.axes.xaxis.set_ticklabels([])
ax2.axes.xaxis.set_ticklabels([])
#ax3.axes.xaxis.set_ticklabels(np.arange(30,51,1))
ax2.set_ylabel(r'Algae $[\%]$')
ax3.set_ylabel(r'Mussels $[\%]$')
ax3.set_xlabel(r'Time $[years]$')
ax1.set_xlim([trim/365,tspan[-1]/365])
ax2.set_xlim([trim/365,tspan[-1]/365])
ax3.set_xlim([trim/365,tspan[-1]/365])
ax1.set_ylim([-10,110])
ax2.set_ylim([-10,110])
ax3.set_ylim([-10,110])
# FFT
forcing = m.Ft(tspan)
#print(len(forcing))
trimfft = int(len(forcing)*(2.0/5.0)) # the index from which to trim the time series to clean transients
#print(trim)
frq = np.fft.fftfreq(forcing[-trimfft:].size,d=0.01/365)
fft_forcing = np.absolute((np.fft.fft(forcing[-trimfft:])))
fft_signal  = np.absolute((np.fft.fft(result[-trimfft:,0])))
normalize_fft = np.amax(fft_signal[1:])
ax4.plot(frq[1:],fft_forcing[1:]/np.amax(fft_forcing[1:]),'m:',label=r'forcing')
ax4.plot(frq[1:],fft_signal[1:]/normalize_fft,'b',label=r'signal')
ax4.set_xlim([0.1,2.0])
ax4.set_ylim([-0.01,1.2])
ax4.set_xlabel(r'freq $[1/years]$')
peaks=signal.find_peaks(result[:,0])
ax1.stem(tspan[peaks[0]]/365,(result[:,0])[peaks[0]]*100.0)
#ax[0].legend(loc='upper left')
ax4.legend(loc='upper right')
#ax[0].legend(loc='upper left')
#ax[1].legend(loc='upper left')
plt.tight_layout()
#plt.savefig('results/Beninca_Ito_and_fft_Tmax{:3.2f}_ito{:4.3f}'.format(Tmax,ito).replace('.','_')+'.pdf')
#plt.savefig('results/Beninca_Ito_and_fft_Tmax{:3.2f}_ito{:4.3f}'.format(Tmax,ito).replace('.','_')+'.png')

In [ ]:
# Save results to text file
print(tspan.reshape(-1,1).shape,result.shape)
tspan_results=np.hstack((tspan.reshape(-1,1)/365.0,100.0*result))
np.savetxt("../../Dropbox/code/Beninca/ts-tmax27_0-with-tmax.dat",tspan_results)